In [1]:
import os
import json
import csv
import pandas as pd
import numpy as np
import math
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
# results_csv_file = r"/Users/arvin/Documents/ucla research/active learning/results/results_11_13_2023_merged_v10.csv"
# results_csv_file = r"/Users/arvin/Documents/ucla research/active learning/results/results_11_13_2023_merged_v11.csv"
#results_csv_file = r"/Users/arvin/Documents/ucla research/active learning/results/results_11_13_2023_merged_CHAOS_v12.csv"
#results_csv_file = r"/Users/arvin/Documents/ucla research/active learning/results/results_11_13_2023_merged_LVSC_v12.csv"
#results_csv_file = r"/Users/arvin/Documents/ucla research/active learning/results/results_2_20_2024_merged_v13.csv"
results_csv_file = r"/Users/arvin/Documents/ucla research/active learning/results/results_11_16_2023_merged_edited1.csv"

In [3]:
exp_df = pd.read_csv(results_csv_file)  
ignore_string_in_name_lst = []
include_string_in_name_lst = ["CHAOS"]

In [4]:
included_exp_names = exp_df['exp'].tolist()

In [5]:
def get_base_exp_name(exp_name):
    exp_field = "exp"
    exp_field_len_w_index = len(exp_field)+1
    exp_field_index = exp_name.find(exp_field)
    exp_name = exp_name[:exp_field_index]+exp_name[exp_field_index+exp_field_len_w_index:]
    return exp_name

In [6]:
exp_groups = dict()

for exp_name in included_exp_names:
    base_exp_name = get_base_exp_name(exp_name)
    if base_exp_name in exp_groups:
        exp_groups[base_exp_name] += [exp_name]
    else:
        exp_groups[base_exp_name] = [exp_name]

In [7]:
exp_length = 5
num_rounds = 10
round_strings =  ["round" + str(round_num) + "_test" for round_num in range(num_rounds)]
base_exp_names = list(exp_groups.keys())

In [8]:
results_dict = {}

for base_exp_name, exp_group in exp_groups.items():
    continue_flag = False
    for ignore_string_in_name in ignore_string_in_name_lst:
        if ignore_string_in_name in base_exp_name:
            continue_flag = True
            break
    for include_string_in_name in include_string_in_name_lst:
        if include_string_in_name not in base_exp_name:
            print("rejected, ", base_exp_name)
            continue_flag = True
            break
    if continue_flag:
        continue
    exp_dict = {}
    for i, run in enumerate(exp_group):
        sub_df = exp_df.loc[exp_df['exp'] == run]
        for round_string in round_strings:
            val = sub_df[round_string].iloc[0]
            if not math.isnan(val):
                if round_string not in exp_dict:
                    exp_dict[round_string] = [float(val)]
                else:
                    exp_dict[round_string] += [float(val)]
    results_dict[base_exp_name] = exp_dict
    print("accepted ", base_exp_name)

rejected,  DMPLS__coreset_cl_bs8_v5
rejected,  DMPLS__coreset_label2_uncertainty1_v8
rejected,  DMPLS__coreset_slice_pos10_v7
rejected,  DMPLS__coreset_v10
rejected,  DMPLS__coreset_fuse10_extra_loss_wt06_use_slice_pos_v7
rejected,  DMPLS__coreset_fuse10_pos_loss1_wt02_pos_loss2_wt01_use_slice_pos_use_phase_bs9_v7
rejected,  DMPLS__coreset_label2_v7
rejected,  DMPLS_coregcn_bugfix
rejected,  DMPLS_sup_LVSC__coreset_fuse20_n50_pos_loss1_wt02_pos_loss2_wt01_pos_loss3_wt005_use_slice_pos_use_phase_use_patient_uncertainty1_v13
rejected,  DMPLS_sup_LVSC__coreset_v12
rejected,  DMPLS_sup__coregcn_model_fts_ss10_rstart_v10
rejected,  DMPLS_sup__random_v11
rejected,  DMPLS__coreset_cl_bs4_v5
rejected,  DMPLS__coreset_slice_pos10_patient10_v7
rejected,  DMPLS__random_v10
rejected,  DMPLS__vaal_v10
rejected,  DMPLS__coreset_cl_bs8_mdl_preds_v5
rejected,  DMPLS__coreset_preds_v5
rejected,  DMPLS__random_v7
rejected,  DMPLS__coreset_cl_bs8_mdl_ft_v6
rejected,  DMPLS__coreset_uncertainty1_v7
reject

In [9]:
index = pd.Index(list(results_dict.keys()), name="exp_name")
round_counts_strings = [[round_string, f"round{i}_count"] for i, round_string in enumerate(round_strings)]
df = pd.DataFrame(index=index, columns=sum(round_counts_strings, []))

In [10]:
for base_exp_name, results in results_dict.items():
    sub_df = df.loc[base_exp_name]
    results = results_dict[base_exp_name]
    for round_string, count_string in round_counts_strings:
        if round_string in results:
            round_results = results[round_string]
            num_results = len(round_results)
            mean_result = np.mean(round_results)
            std_results = np.std(round_results, ddof=1)
            mean_string = '%.3f' % mean_result
            std_string = '%.3f'% std_results
            sub_df[round_string] = f"{mean_string}+/-{std_string}"
            sub_df[count_string] = num_results

/Users/arvin/.virtualenvs/al/lib/python3.9/site-packages/numpy/core/_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/arvin/.virtualenvs/al/lib/python3.9/site-packages/numpy/core/_methods.py:257: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [11]:
df
df.sort_values(by=["round4_test"], ascending=False)

,round0_test,round0_count,round1_test,round1_count,round2_test,round2_count,round3_test,round3_count,round4_test,round4_count,round5_test,round5_count,round6_test,round6_count,round7_test,round7_count,round8_test,round8_count,round9_test,round9_count
exp_name,,,,,,,,,,,,,,,,,,,,
DMPLS_sup_CHAOS__coreset_fuse20_pos_loss1_wt02_pos_loss2_wt01_use_slice_pos_use_phase_uncertainty1_v13,0.686+/-0.156,3,0.771+/-0.059,3,0.766+/-0.033,3,0.839+/-0.039,3,0.806+/-0.032,3,0.887+/-0.016,3,0.883+/-0.037,3,0.910+/-0.035,3,0.962+/-0.004,3,NaN,NaN
DMPLS_sup_CHAOS__coreset_fuse10_pos_loss1_wt02_pos_loss2_wt01_use_slice_pos_use_phase_uncertainty1_v13,0.640+/-nan,1,0.642+/-nan,1,0.809+/-nan,1,0.743+/-nan,1,0.803+/-nan,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DMPLS_sup_CHAOS_CT__random_v13,0.787+/-0.093,3,0.791+/-0.118,3,0.751+/-0.129,3,0.747+/-0.042,3,0.803+/-0.097,3,0.889+/-0.030,3,0.873+/-0.016,3,0.904+/-0.025,3,0.950+/-0.017,3,NaN,NaN
DMPLS_sup_CHAOS__coreset_v13,0.749+/-0.057,3,0.797+/-0.100,3,0.780+/-0.044,3,0.777+/-0.031,3,0.799+/-0.075,3,0.831+/-0.043,3,0.899+/-0.013,3,0.948+/-0.004,3,0.939+/-0.022,3,NaN,NaN
DMPLS_sup_db_bald_CHAOS__v13,0.741+/-0.100,3,0.662+/-0.006,3,0.696+/-0.043,3,0.760+/-0.027,3,0.789+/-0.050,3,0.894+/-0.007,3,0.895+/-0.010,3,0.945+/-0.015,3,0.947+/-0.013,3,NaN,NaN
DMPLS_sup_CHAOS_CT__random_v12,0.479+/-0.171,3,0.634+/-0.086,3,0.705+/-0.058,3,0.658+/-0.111,3,0.740+/-0.029,3,0.753+/-0.029,3,0.755+/-0.018,3,0.766+/-0.015,3,0.771+/-0.005,3,NaN,NaN
DMPLS_sup_CHAOS__coreset_fuse20_pos_loss1_wt02_pos_loss2_wt01_use_slice_pos_use_phase_uncertainty1_v13_old,0.389+/-0.054,3,0.500+/-0.144,3,0.577+/-0.041,3,0.576+/-0.062,3,0.710+/-0.057,3,0.736+/-0.033,3,0.733+/-0.030,3,0.727+/-0.046,3,0.756+/-0.006,3,NaN,NaN
DMPLS_sup_CHAOS__ensemble_var_v13,0.734+/-0.119,3,0.704+/-0.097,3,0.661+/-0.030,3,0.682+/-0.020,3,0.707+/-0.034,3,0.725+/-0.044,3,0.841+/-0.080,3,0.812+/-0.055,3,0.885+/-0.032,3,NaN,NaN
DMPLS_sup_CHAOS__coreset_v12,0.360+/-0.075,3,0.546+/-0.021,3,0.659+/-0.023,3,0.644+/-0.061,3,0.701+/-0.040,3,0.731+/-0.040,3,0.748+/-0.013,3,0.763+/-0.022,3,0.787+/-0.014,3,NaN,NaN
